## CARES act college stimulus by ownership form

By Charlie Eaton, April 2, 2020

Note: Ownership data is ownership as of 2017 and may have shifted since then. Publicly traded firms may have a private equity investment indicator of "1" if there is no publicly available information about private equity exits from the investment.

In [ ]:
quietly {
cd "/Users/Charlie/Desktop"
import excel using ///
"3_30_20---resort_03-24-20_estimated_higher_ed_fund_distribution_620pm.xlsx" ///
, sheet("Insts no fully online") firstrow case(lower) clear
drop if opeid==.
tostring opeid, gen(opeidstring)
replace opeidstring="0"+opeidstring if strlen(opeidstring)==6
replace opeidstring="0"+opeidstring if strlen(opeidstring)==7
replace opeidstring=substr(opeidstring,1,6)
destring opeidstring, gen(opeid6)
encode schooltype, gen(schooltype_n)
keep if schooltype_n==2
*ds, varwidth(28)
*summarize
save stimulusbyschoolandfirm, replace
}

In [ ]:
quietly {
use "/Users/Charlie/Dropbox/Asymmetry by Design/Data/d_OPEID_Asymmetry.dta", clear
bysort opeid: egen maxyear=max(year)
keep if maxyear==year
rename year ownershipyear
rename opeid opeid6
rename opeidstring opeidstringeaton
collapse (first) opeidstringeaton pe_ind public control_n iclevel institutionname ///
ownershipyear systemid systemnm (sum) claimcount, by(opeid6)
}
merge 1:1 opeid6 using stimulusbyschoolandfirm, gen(owner_stimulus_merge) keep(2 3)

In [ ]:
quietly gen ownership="other"
quietly replace ownership="private equity" if pe_ind==1
quietly replace ownership="publicly traded" if public==1

In [ ]:
format totaldollarsremovingonlineo pelldollarsremovingonlineon %12.0fc
quietly replace systemnm = institutionname if systemnm==""
quietly outsheet using collegestimulusschoolowner.csv, comma

In [ ]:
table ownership, c(sum totaldollarsremovingonlineo) format(%12.0fc)
table ownership, c(sum pelldollarsremovingonlineon) format(%12.0fc)

In [ ]:
collapse (first) opeidstringeaton pe_ind public ownership control_n iclevel institutionname ///
ownershipyear systemnm (sum) claimcount totaldollarsremovingonlineo pelldollarsremovingonlineon ///
, by(systemid)
replace systemnm="Paul Mitchell franchises" if systemid==454944

In [ ]:
gsort -totaldollarsremovingonlineo
quietly outsheet using collegestimulusfirmowner.csv, comma

In [ ]:
quietly set linesize 200
list systemid systemnm ownership pe_ind totaldollarsremovingonlineo pelldollarsremovingonlineon if _n<11